In [1]:
import pandas as pd
!conda install -c conda-forge/label/cf201901 wikipedia
import wikipedia as wp

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge/label/cf201901
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge/label/cf201901
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge/label/cf201901
    wikipedia-1.4.0            |           py35_0          20 KB  conda-forge/label/cf201901
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

    wikipedia:       1.4.0-py35_0      conda-forge/label/cf201901

The following packages will be UPDATED:

    certifi:         2018.8.24-py35_1                

## Read data from wikipedia

In [2]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
Toronto_df = pd.read_html(html)[0]
Toronto_df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


## Renaming Header

In [3]:
Toronto_df.columns=Toronto_df.iloc[0]
Toronto_df.head()

,Postcode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [4]:
Toronto_df=Toronto_df.reindex(Toronto_df.index.drop(0))
Toronto_df.head()
Toronto_df.shape

(288, 3)

## Exclude the records where Borough=Not assigned, then reindexing the dataframe

In [5]:
Toronto_database=Toronto_df[Toronto_df['Borough']!='Not assigned']
Toronto_database=Toronto_database.reset_index()
Toronto_database=Toronto_database.drop('index',1)
Toronto_database.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Consider Borough in Neighbourhood where Neighbourhood=Not assigned

In [6]:
Toronto_database1=Toronto_database[Toronto_database['Neighbourhood']=='Not assigned']
Toronto_database1=Toronto_database1.drop('Neighbourhood',1)
Toronto_database1['Neighbourhood']=Toronto_database1['Borough']
Toronto_database1

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Queen's Park


In [7]:
Toronto_database=Toronto_df[Toronto_df['Neighbourhood']!='Not assigned']
Toronto_database=Toronto_database.append(Toronto_database1)
Toronto_database.shape

(211, 3)

## Merge Neighbourhood where based on duplicate Postcode

In [8]:

TDF=Toronto_database.groupby('Postcode').agg({
                'Borough': lambda x: x.unique(),
                'Neighbourhood': lambda x: ', '.join(x)
    })

TDF=TDF.reset_index()
TDF

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


## Final Table size

In [9]:
TDF.shape

(103, 3)

In [10]:
geodf=pd.read_csv('http://cocl.us/Geospatial_data')

geodf = geodf.rename(columns = {"Postal Code": "Postcode", 
                                  "Latitude":"Latitude", 
                                  "Longitude": "Longitude"}) 
geodf.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
Toronto_DB = pd.merge(TDF,
                 geodf[['Postcode','Latitude', 'Longitude']],
                 on='Postcode')

Toronto_DB=Toronto_DB[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
Toronto_DB

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## import required library

In [12]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from geopy.geocoders import Nominatim
import requests
import json
from pandas.io.json import json_normalize
import numpy as np

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

## Creating Toronto map with Neighbourhood

In [13]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6523873, -79.3835641.


In [14]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_Toronto

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_DB['Latitude'], Toronto_DB['Longitude'], Toronto_DB['Borough'], Toronto_DB['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

## FIlter data for North York and createing NorthYork DataFrame for Analysis

In [15]:
NorthYork_data = Toronto_DB[Toronto_DB['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [16]:
address = 'North York, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.7708175, -79.4132998.


In [17]:
# create map of New York using latitude and longitude values
map_NorthYork = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, Neighbourhood in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Borough'], NorthYork_data['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NorthYork)  
    
map_NorthYork

## Using Forusquare App to collect the near by vanues detail for "Hillcrest Village"

In [18]:
CLIENT_ID = '2HJUONTVXJVO3S0M4DBWEDOJBYYWHW4RYR0B04OIT3STCFCB' # your Foursquare ID
CLIENT_SECRET = 'ER505YC0NL5G4EOGQQUK5QJOZ30QAXEQD3T2A3MCMSHNFBCO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
NorthYork_data.loc[0, 'Neighbourhood']

'Hillcrest Village'

In [19]:
neighbourhood_latitude = NorthYork_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = NorthYork_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = NorthYork_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


In [20]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighbourhood_latitude, neighbourhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,New York Fries,Fast Food Restaurant,43.803664,-79.363905
2,AY Jackson Pool,Pool,43.804515,-79.366138
3,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
4,Duncan Creek Park,Dog Run,43.805539,-79.360695


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighbourhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude']
                                  )
NorthYork_venues.head()

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


## will analyse the "Hillcrest Village" vanue detail for creating map with venue

In [25]:
NorthYork_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
"CFB Toronto, Downsview East",2,2,2,2,2,2
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


In [26]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [27]:
# one hot encoding
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
NorthYork_onehot['Neighbourhood'] = NorthYork_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
NorthYork_onehot.shape

(241, 108)

In [30]:
NorthYork_grouped = NorthYork_onehot.groupby('Neighbourhood').mean().reset_index()
NorthYork_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,...,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview, North Park, Upwood Park",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery, Humberlea",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
NorthYork_grouped.shape

(23, 108)

## will consider only top 5 venues of neghbourhood for clustering

In [33]:
num_top_venues = 5

for hood in NorthYork_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = NorthYork_grouped[NorthYork_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
              venue  freq
0       Coffee Shop  0.11
1  Sushi Restaurant  0.06
2        Restaurant  0.06
3    Sandwich Place  0.06
4     Shopping Mall  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.09
1  Fast Food Restaurant  0.09
2           Coffee Shop  0.09
3         Grocery Store  0.05
4        Sandwich Place  0.05


----CFB Toronto, Downsview East----
           venue  freq
0        Airport   0.5
1           Park   0.5
2   Liquor Store   0.0
3  Moving Target   0.0
4  Movie Theater   0.0


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.17
1  Caribbean Restaurant  0.17
2   Japanese Restaurant  0.17
3        Baseball Field  0.17
4      Baske

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = NorthYork_grouped['Neighbourhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Shopping Mall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Indian Restaurant,Café,Liquor Store,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Electronics Store,Convenience Store,Cosmetics Shop


## will consider K value 5 for clustering

In [36]:
# set number of clusters
kclusters = 5

NorthYork_grouped_clustering = NorthYork_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NorthYork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 4], dtype=int32)

In [37]:
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Shopping Mall
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Indian Restaurant,Café,Liquor Store,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto, Downsview East",Airport,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Electronics Store,Convenience Store,Cosmetics Shop
5,Downsview Central,Korean Restaurant,Food Truck,Business Service,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
6,Downsview Northwest,Grocery Store,Athletics & Sports,Discount Store,Liquor Store,Empanada Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
7,Downsview West,Grocery Store,Bank,Shopping Mall,Moving Target,Frozen Yogurt Shop,Fried Chicken Joint,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,"Downsview, North Park, Upwood Park",Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega
9,"Emery, Humberlea",Baseball Field,Women's Store,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


In [38]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NorthYork_merged = NorthYork_data

In [39]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
NorthYork_merged = NorthYork_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
NorthYork_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Cosmetics Shop,Japanese Restaurant,Bakery,Women's Store,Toy / Game Store,Food Court
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2.0,Cafeteria,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
NorthYork_merged['Cluster Labels']=NorthYork_merged['Cluster Labels'].apply(lambda x: int(x) if x == x else 0)
NorthYork_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Cosmetics Shop,Japanese Restaurant,Bakery,Women's Store,Toy / Game Store,Food Court
2,M2K,North York,Bayview Village,43.786947,-79.385975,1,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2,Cafeteria,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating Cluster map

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NorthYork_merged['Latitude'], NorthYork_merged['Longitude'], NorthYork_merged['Neighbourhood'], NorthYork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

# Cluster1

In [42]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 0, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,North York,0,Park,Convenience Store,Bank,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
8,North York,0,Park,Food & Drink Shop,Fast Food Restaurant,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
13,North York,0,Airport,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


# Cluster2

In [43]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 1, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Dim Sum Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
1,North York,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Cosmetics Shop,Japanese Restaurant,Bakery,Women's Store,Toy / Game Store,Food Court
2,North York,1,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Empanada Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
5,North York,1,Ramen Restaurant,Restaurant,Coffee Shop,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Café,Movie Theater,Plaza,Pizza Place
7,North York,1,Grocery Store,Discount Store,Pharmacy,Pizza Place,Butcher,Coffee Shop,Food Truck,Food Court,General Entertainment,Furniture / Home Store
9,North York,1,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Electronics Store,Convenience Store,Cosmetics Shop
10,North York,1,Coffee Shop,Gym,Asian Restaurant,Beer Store,Grocery Store,Sandwich Place,Clothing Store,Chinese Restaurant,General Entertainment,Café
11,North York,1,Coffee Shop,Frozen Yogurt Shop,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Diner,Restaurant,Shopping Mall
12,North York,1,Miscellaneous Shop,Massage Studio,Bar,Falafel Restaurant,Coffee Shop,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop
14,North York,1,Grocery Store,Bank,Shopping Mall,Moving Target,Frozen Yogurt Shop,Fried Chicken Joint,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


# Cluster3

In [45]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 2, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,2,Cafeteria,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


# Cluster4

In [46]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 3, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North York,3,Empanada Restaurant,Pizza Place,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


# Cluster5

In [47]:
NorthYork_merged.loc[NorthYork_merged['Cluster Labels'] == 4, NorthYork_merged.columns[[1] + list(range(5, NorthYork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,4,Baseball Field,Women's Store,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
